## Part 0: Prologue

Итак, собственно, подходит к концу соревнование. Всегда любил наблюдать за лидербордами в последние 24 часа. Они, по сути, "вскипают" и в его середине, а иногда и в верхней части происходят бурные трансформации. А если данные разбиты в сильно разных пропорциях, или сильно отличаются по характеристикам то бывает ещё и сильный шафл. Страшно, но бесценно. 

Впрочем, сейчас не об этом. Этот кернел пишется за сутки до надвигающихся событий, а потому просто опишу своё решение и пожелаю всем удачи. 

Если описать кратко, то всё довольно просто. Построим рекомендательный алгоритм на тех юзерах, о которых мы что-то знаем, а остальным ~~дадим пиццу, ведь её все любят~~ предложим что-то просто популярное. Звучит как план, скажи же, Джеффри...

![Lebowski](https://i.postimg.cc/cHGZVC6K/That-s-a-great-plan-Walter-That-s-fuckin-ingenious-if-I-understand-it-correctly-It-s-a-Swiss-fu.png)

## Part I. Preparation 

Собственно, начнём. 

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Загрузим данные и импортируем нужные библиотеки

In [ ]:
!mkdir cookbook

In [ ]:
!cp /kaggle/input/cookbook/recsys.py cookbook/recsys.py
!cp /kaggle/input/cookbook/generic_preprocessing.py cookbook/generic_preprocessing.py

In [ ]:
import numpy as np 
import pandas as pd 

# Несколько утилиток для предпроцессинга данных
from cookbook.recsys import * 
from cookbook.generic_preprocessing import * 

import scipy
import random
import scipy.sparse as sp
from itertools import cycle, islice
from implicit.nearest_neighbours import BM25Recommender
from datetime import date
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)

 В - Воспроизводимость. Зафиксируем все сиды, чтобы гарантировать неслучайную работу псевдослучайного ГСЧ.

In [ ]:
def seed_everything(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


In [ ]:
seed_everything(1000)

Книг слишком много(на самом деле книг много не бывает). Но тем не менее возьмём топ наиболее популярных

In [ ]:

BOOK_CNT = 4000

# Директории с данными
DATA_DIR = "/kaggle/input/mts-ml-summer-school/"

SUB_DIR = "/kaggle/input/cookbook/"

In [ ]:
items = pd.read_csv(os.path.join(DATA_DIR, "items.csv"))
users = pd.read_csv(os.path.join(DATA_DIR, "users.csv"))
interactions = pd.read_csv(os.path.join(DATA_DIR, "interactions.csv"))

# Топ за 30 дней. Нужен в качестве примера сабмита
sub = pd.read_csv(os.path.join(SUB_DIR, "submission_pop_30.csv"))


In [ ]:
full_set = pd.merge( 
    pd.merge(interactions, items, how='left', left_on="item_id", right_on="id"),
    users, how="left", left_on="user_id", right_on="user_id").drop(["id"], axis=1)

## Part II. Preprocessing


Для начала нужно немного трансформировать исходные данные, обработать пропуски и вот это всё. Здесь стоит сделать одну ремарку. Далее пойдёт только то, что связано с тем, что зашло. Потому что перепробовано слишком много и ещё больше даже не проверено. 

In [ ]:
full_set.age.fillna("unknown_age", inplace=True)
full_set.sex.fillna("unknown_sex", inplace=True)

In [ ]:
full_set['age'] = full_set.age.apply(lambda x: x.split("_")[0])

Просто получение последнего автора (для наглядности)

In [ ]:
def get_info_last(x, word):
    try: 
        x = x.replace('\n', '').replace('\t', '').replace('  ', ' ').replace('/', ',').lower()
        return x.split(',')[-1].strip().rstrip()
    except Exception as e:
        return word

In [ ]:
full_set['one_author'] = full_set.authors.apply(lambda x: get_info_last(x, 'неизвестен'))


In [ ]:
full_set.start_date = pd.to_datetime(full_set.start_date)
full_set['init_date'] = "2017-12-31"
full_set.init_date = pd.to_datetime(full_set.init_date)

In [ ]:
full_set['diff'] = full_set.start_date -  full_set.init_date

full_set['weight'] = full_set['diff'].apply(lambda x: 1.2578890000000001e-46*pow(x.days,17.08534) )

Идея преобразования выше состоит в следующем. Возьмём количество дней прошедших к 31 декабря 2017 года и взвесим нелинейной функцией, таким образом, взаимодействия в последний месяц будут иметь значительно большый и быстрее растущий вес, чем за период до декабря, а в последнюю неделю и того значительнее. 

Чем ближе к 31 декабря наблюдение, тем важнее для нас предсказать его правильно. 

In [ ]:
x = np.array(range(730))
y = 1.2578890000000001e-46*(x**17.08534)

plt.plot(x,y,label='y = 1.257889e-46*x^17.08534')
plt.title('Вес в зависимости от дня')
plt.grid()
plt.legend()
plt.show()


In [ ]:
users_ids = sub.Id.tolist()

In [ ]:
# Возьмём первые BOOK_CNT книг по встречаемости

items_list = full_set[full_set.start_date  > "2019-10-30"].groupby('item_id')['progress'].count().reset_index().sort_values(by='progress', ascending=False)['item_id'][:BOOK_CNT]

In [ ]:
# Для модели оставим только их

interactions_existing = full_set[full_set.item_id.isin(items_list)]

## Part III. Simple and curious EDA (The second part solution)

Вообще, начнём наверное с конца, а именно с того, чем будем заполнять пропуски в сабмите. Тут всё довольно просто и одновременно интересно. Все указаннные ID предподсчитаны за месяц для разных групп отдельно, чтобы не захлямлять код здесь. 

In [ ]:
_temp = full_set[full_set.start_date >= "2019-12-01"]

In [ ]:
def get_items_info(sex_type, book_list):
    
    print("_"*100)
    print(sex_type) 
    for i in book_list:
        print(items[items.id ==i]['title'].values[0])

        
def get_items_info_by_sex_and_age(sex, age):
    _temp[(_temp.sex==1.0) & (_temp.age=="18") ].groupby(['item_id'])['user_id'].count().reset_index().sort_values(by=['user_id'], ascending=[False]).head(10)['item_id'].tolist()

In [ ]:
print(get_items_info("Unknown sex:", [276903, 168963, 187325, 352049, 79499, 267817, 33801, 283713, 93751, 50718]))  
print(get_items_info('Male sex', [283713, 184549, 276903, 357309, 55466, 385281, 143175, 168963, 352049, 287060]))  
print(get_items_info('Female sex', [283713, 184549, 143175, 168963, 80003, 357309, 56877, 55466, 276903, 385281]))  

In [ ]:
def get_books_by_stats(desc, sex=1.0, age="18"):
    
    
    top_items = _temp[(_temp.sex==sex) & (_temp.age==age) ].groupby(['item_id'])['user_id'].count().reset_index().sort_values(by=['user_id'], ascending=[False]).head(10)['item_id'].tolist()
    return get_items_info(desc, top_items) 


Посмотрим в срезе возраста. Для начала посмотрим по женщинам.

In [ ]:
print(get_books_by_stats("Women 18", sex=1.0, age="18"))

In [ ]:
print(get_books_by_stats("Women 65", sex=1.0, age="65"))

И по мужчинам

In [ ]:
print(get_books_by_stats("Men 18", sex=1.0, age="18"))

In [ ]:
print(get_books_by_stats("Men 65", sex=1.0, age="65"))

Очевидно, что в какой-то момент, людям становится не очень интересно двигаться вперёд, ~~доминировать, властвовать, унижать~~ убеждать и воздействовать. Является ли причиной этого достижение всего, или потеря желания, вопрос дискуссионный.

Кроме того в ТОПе очень часто появляются представители своего рода книжного "арт-хауса", вроде произведений братьев Стругацких "Пикник на обочине" и "Понедельник начинается в субботу". Это те, кто писали о сталкерах до того, как это стало мейнстримом, до ребят из GSC Game World, да и до самой аварии на ЧАЭС. **Press F to pay respect.** ~~То что водка спасает от радиации, они там не писали.~~ 



Cкорее всего это связано с наличием тематических подборок в самом приложении, что тоже к слову, открывает пространство для маневров и изысканий в этом направлении. 

![Screen](https://i.postimg.cc/7ZG9V6wj/photo-2021-05-16-22-54-03.jpg)

Собственно, мораль. Наверно лучше будет разделить популярные книги по полу и возрасту. А почему бы собственно и нет.

In [ ]:
conditions = { (0.0, "18"):[80003, 283713, 287060, 184549, 267817, 385281, 56877, 262464, 168963, 131612],
              (1.0, "18"):[184549, 276903, 80003, 385281, 55466, 283713, 112869, 168963, 364570, 264133],
              (0.0, "25"):[80003, 56877, 262464, 184549, 357309, 168963, 276903, 385281, 143175, 287060],
              (1.0, "25"):[385281, 283713, 276903, 357309, 187325, 229030, 287060, 184549, 168963, 112869],
              (0.0, "35"):[283713, 184549, 357309, 80003, 56877, 168963, 302067, 242176, 385281, 344047],
              (1.0, "35"):[184549, 357309, 283713, 385281, 276903, 112869, 287060, 5408, 143175, 168963],
              (0.0, "45"):[283713, 184549, 143175, 321351, 168963, 55466, 357309, 323949, 190198, 112869],
              (1.0, "45"):[283713, 184549, 357309, 276903, 55466, 168963, 50718, 143175, 246948, 242176],
              (0.0, "55"):[283713, 143175, 168963, 55466, 184549, 160349, 357309, 323949, 190198, 51423],
              (1.0, "55"):[283713, 184549, 352049, 55466, 143175, 276903, 58480, 51581, 112869, 40953],
              (0.0, "65"):[283713, 143175, 184549, 55466, 374648, 160349, 168963, 267817, 178529, 352049],
              (1.0, "65"):[283713, 55466, 51423, 143526, 276903, 126630, 184549, 232758, 143175, 49054],
              
              (1.0):[283713, 184549, 276903, 357309, 55466, 385281, 143175, 168963, 352049, 287060],
              (0.0):[283713, 184549, 143175, 168963, 80003, 357309, 56877, 55466, 276903, 385281],
              ('unknown_sex'):[276903, 168963, 187325, 352049, 79499, 267817, 33801, 283713, 93751, 50718]
            }

На этом с тем, чем будем заполнять вроде бы разобрались. 

## Part IV. BM25Recommender + Prediction

Вообще, я честно долго пытался сварить  LightFM, но что-то пошло не так. 

In [ ]:
users_inv_mapping = dict(enumerate(interactions_existing['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
print(len(users_mapping))


items_inv_mapping = dict(enumerate(interactions_existing['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
print(len(items_mapping))

In [ ]:
def get_coo_matrix(df, 
                   user_col='user_id', 
                   item_col='item_id', 
                   weight_col=None, 
                   users_mapping=users_mapping, 
                   items_mapping=items_mapping):
    if weight_col is None:
        weights = np.ones(len(df), dtype=np.float32)
    else:
        weights = df[weight_col].astype(np.float32)

    interaction_matrix = sp.coo_matrix((
        weights, 
        (
            df[user_col].map(users_mapping.get), 
            df[item_col].map(items_mapping.get)
        )
    ))
    return interaction_matrix

In [ ]:
train = get_coo_matrix(interactions_existing, weight_col='weight').tocsr()

In [ ]:
b25_model = BM25Recommender(K=10, K1=2.0, B=0.75)
b25_model.fit(train.T)

Параметры позаимствованы отсюда   [BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)

In [ ]:
top_N = 10
user_id = sub['Id'].iloc[2]
row_id = users_mapping[user_id]
print(f'Рекомендации для пользователя {user_id}, номер строки - {row_id}')


In [ ]:
def generate_implicit_recs_mapper(model, train_matrix, N, user_mapping, item_inv_mapping):
    def _recs_mapper(user):
        user_id = user_mapping[user]
        recs = model.recommend(user_id, 
                               train_matrix, 
                               N=N, 
                               filter_already_liked_items=True)
        return [item_inv_mapping[item] for item, _ in recs]
    return _recs_mapper


In [ ]:
mapper = generate_implicit_recs_mapper(b25_model, train, top_N, users_mapping, items_inv_mapping)

In [ ]:
%time
recs = pd.DataFrame({
    'user_id': interactions_existing['user_id'].unique()
})
recs['item_id'] = recs['user_id'].map(mapper)
recs.head()


Таким образом, удалось получить предсказания по следующему количеству пользователей

In [ ]:
recs[recs.user_id.isin(sub.Id) ].shape[0]

In [ ]:
missing_ids = sub[~sub.Id.isin(recs[recs.user_id.isin(sub.Id)]['user_id'])]['Id'].tolist()

In [ ]:
len(missing_ids)

In [ ]:
recs_metric = recs.explode('item_id')
recs_metric['rank'] = recs_metric.groupby('user_id').cumcount() + 1
recs_metric.head(top_N + 2)

In [ ]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    
    #print(result)
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

In [ ]:
print(compute_metrics(interactions_existing, recs_metric, top_N))

In [ ]:
res = recs[recs.user_id.isin(sub.Id)]
recs_dict =  {i[0]:i[1] for  i in res[['user_id', 	'item_id']].values}


## Part V. Blend it all

In [ ]:

class Restoring:
    """Restore missing values. Just filling by common according user's features"""
    def __init__(self):
        self.users = users
        self.users.age.fillna("unknown_age", inplace=True)
        self.users.sex.fillna("unknown_sex", inplace=True)
        self.users['age'] = users.age.apply(lambda x: x.split("_")[0])
        self.conditions = conditions
        self.missing_ids = missing_ids
        self.conditions_cnt = {}

    def get_users_sex(self, user_id):
        if len(self.users[self.users.user_id == user_id]) > 0:
            return self.users[self.users.user_id == user_id]['sex'].values[0]
        else:
            return None
    
    def get_users_age(self, user_id):
        if len(self.users[self.users.user_id == user_id]) > 0:
            return self.users[self.users.user_id == user_id]['age'].values[0]
        else:
            return None
    
    def restore_values(self, x):

        key = None
        sex = self.get_users_sex(x['Id'])
        age = self.get_users_age(x['Id'])  

        ## Формируем ключ для извлечения популярного значения
        if age in [None, 'age'] or sex in [None, 'unknown_sex']:
            if sex in [None, 'unknown_sex']:
                key = ('unknown_sex')
            else:
                key = (sex)

        else:
            key = (sex, age)  
        
        ## Если нет предсказания добавим популярное
        if x['Id'] in self.missing_ids:

            if key in self.conditions_cnt.keys():
                self.conditions_cnt[key] +=1
            else:
                self.conditions_cnt[key] = 1
            
            return ' '.join([str(i) for i in self.conditions[key]])
        else:
            ##  Если есть трансформируем его в строку
            if x['Id'] in recs_dict.keys():
                return ' '.join([str(i) for i in recs_dict[x['Id']]])
            else:
            ## Или оставим всё как есть
                return x['Predicted']
    
    def clear_dict(self):
        self.conditions_cnt = {}
        

In [ ]:
rest = Restoring()

In [ ]:
sub2 = sub.copy()

In [ ]:
sub2['Predicted'] = sub2.apply(lambda x: rest.restore_values(x), axis=1)

In [ ]:
sub2.to_csv("submission.csv", index=None)

In [ ]:
sub2

Количество замен

In [ ]:
rest.conditions_cnt

## Part VI. Epilogue

Вроде всё. Здесь должно быть что-то важное, но ~~важный и вычурный~~ эпилог я придумать не смог. Ну разве что совет любить то, что делаешь и читать хорошие книги. Всем *peace*. 

by MEMPHIS